[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DoranLyong/Awesome-Tensor-Architecture/blob/main/pytorch_reference/simple_reference/06_PyTorch_Acceleration_and_Optimization/03_Hyperparameter_Tuning.ipynb)

# Model Optimization
#### Three areas of optimization: 
1. hyperparameter tuning 
2. quantization
3. pruning

In [3]:
!pip install tensorboardx 
!pip install ray

     |████████████████████████████████| 124 kB 8.8 MB/s 
     |████████████████████████████████| 54.4 MB 12.4 MB/s 
     |████████████████████████████████| 69 kB 5.0 MB/s 
     |████████████████████████████████| 118 kB 11.8 MB/s 
     |████████████████████████████████| 4.0 MB 11.1 MB/s 
     |████████████████████████████████| 701 kB 11.2 MB/s 
     |████████████████████████████████| 302 kB 11.1 MB/s 
     |████████████████████████████████| 84 kB 2.6 MB/s 
     |████████████████████████████████| 118 kB 11.2 MB/s 


In [25]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F 
from torch.utils.data import random_split
import torchvision 
import torchvision.transforms as T 

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Hyperparameter Tuning - feat. ```Ray Tune``` (p.171)
```Ray Tune``` supports SOTA hyperparameter search algorithms and distributed training.

In order to use ```Ray Tune```: 
1. Define our hyperparameters and their search space.
2. Write a function to wrap our training loop.
3. Run ```Ray Tune``` hyperparameter tuning.

In [12]:
# Design our model 

class Net(nn.Module):
    def __init__(self, nodes_1=120, nodes_2=84):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=nodes_1) # configure nodes_1 for Ray Tune 
        self.fc2 = nn.Linear(nodes_1, nodes_2)  # configure nodes_2 for Ray Tune 
        self.fc3 = nn.Linear(nodes_2, 10)  # classifier 

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        assert x.dim() == 4, f"Input tensor to temporal convolution must be 4d! but, {x.dim()}d tensor is given"
        
        x = self.pool(F.relu(self.conv1(x))) 
        x = self.pool(F.relu(self.conv2(x))) 
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x 

two hyperparameters, ```nodes_1``` and ```nodes_2```.

In [9]:
# check model 
x = torch.randn(4, 3, 32, 32)

model = Net()
scores = model(x)

two more hyperparameters, ```lr``` and ```batch_size```. <br/>
we can vary the __learning rate__ and __batch size__ used in our training. 

```tune.sample_from()``` and a ```lambda``` function to define a search space. (p. 173)

In [15]:
# import the ray package and define the hyperparameter configuration:

import numpy as np 
from ray import tune 


config = {  "nodes_1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
            "nodes_2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
                 "lr": tune.loguniform(1e-4, 1e-1),
         "batch_size": tune.choice([2, 4, 8, 16])  
        }

In [18]:
a = lambda _: 2 ** np.random.randint(2, 9)

In [22]:
a(_)

16

### Load dataset 

In [24]:
def load_data(data_dir="./data"):
    train_transforms = T.Compose([  T.RandomCrop(32, padding=4),
                                    T.RandomHorizontalFlip(),
                                    T.ToTensor(),
                                    T.Normalize(mean=(0.4914, 0.4822, 0.4465),
                                                std=(0.2023, 0.1994, 0.2010))
                                ])

    test_transforms = T.Compose([   T.ToTensor(),
                                    T.Normalize(mean=(0.4914, 0.4822, 0.4465),
                                                std=(0.2023, 0.1994, 0.2010))
                                ])

    trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, 
                                            download=True, transform=train_transforms
                                            )

    testset = torchvision.datasets.CIFAR10( root=data_dir, train=False, 
                                            download=True, transform=test_transforms
                                            )
    return trainset, testset

### Wrap our training loop into a function (p.174)

In [27]:
def train_model(config):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = Net(config['nodes_1'], config['nodes_2']).to(device=device) # Make the model layer configurable 

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(  model.parameters(),
                            lr=config['lr'],
                            momentum=0.9) # Make the learning rate configurable 

    trainset, testset = load_data()

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(trainset, [test_abs, len(trainset) - test_abs])

    trainloader = torch.utils.data.DataLoader(  train_subset,
                                                batch_size=int(config["batch_size"]), # Make the batch size configurable 
                                                shuffle=True) 

    valloader = torch.utils.data.DataLoader(    val_subset,
                                                batch_size=int(config["batch_size"]), # Make the batch size configurable 
                                                shuffle=True) 

    n_epochs = 10 
    for epoch in range(n_epochs):
        
        # === Start Trainng === # 
        model.train()
        train_loss = 0.0
        epoch_steps = 0

        for data in trainloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # === Start Validation === #
        model.eval()
        val_loss = 0.0
        total = 0
        correct = 0

        for data in valloader:

            with torch.no_grad():
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)

                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()

        print(  f'epoch: {epoch} ',
                f'train_loss: {train_loss / len(trainloader)}',
                f'val_loss: {val_loss / len(valloader)}',
                f'val_acc: {correct / total}')
        
        tune.report(loss = (val_loss / len(valloader)),
                    accuracy = correct / total)

### Determine the ```scheduler``` and the ```reporter``` (p. 176)
* ```scheduler``` : how __Ray Tune__ searchs and selects the hyperparameters.
* ```reporter``` : how we'd like to view the results. 

In [30]:
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


scheduler = ASHAScheduler(  metric="loss",
                            mode="min",
                            max_t=10,
                            grace_period=1,
                            reduction_factor=2) # asynchronous successive halving algorithm (ASHA) for hyperparameter searchs 
                                                # mode="min" -> instruct it to minimize loss. 

reporter = CLIReporter( metric_columns=["loss", "accuracy", "training_iteration"])  # to report the loss, accuracy, training iteration, and selected hyperparameters 
                                                                                    # on the CLI for each run 

### Run ```Ray Tune``` using the ```run()``` method 
* We provision the resources and specify the configuration. 
* We pass in our configuration dictionary, specify the number of samples or runs, 
* and pass in our __scheduler__ and __reporter__ functions. 

In [31]:
from functools import partial

result = tune.run(  partial(train_model),
                    resources_per_trial={"cpu": 2, "gpu": 1},
                    config=config,
                    num_samples=10,
                    scheduler=scheduler,
                    progress_reporter=reporter
                )

2021-11-18 12:25:04,446	WARNING experiment.py:271 -- No name detected on trainable. Using DEFAULT.
2021-11-18 12:25:04,448	INFO registry.py:69 -- Detected unknown callable for trainable. Converting to class.
2021-11-18 12:25:04,449	WARNING function_runner.py:563 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


== Status ==
Current time: 2021-11-18 12:25:04 (running for 00:00:00.22)
Memory usage on this node: 8.4/62.5 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 0/28 CPUs, 0/2 GPUs, 0.0/33.67 GiB heap, 0.0/16.83 GiB objects (0.0/1.0 accelerator_type:TITAN)
Result logdir: /home/milky/ray_results/DEFAULT_2021-11-18_12-25-04
Number of trials: 10/10 (10 PENDING)
+---------------------+----------+-------+--------------+-------------+-----------+-----------+
| Trial name          | status   | loc   |   batch_size |          lr |   nodes_1 |   nodes_2 |
|---------------------+----------+-------+--------------+-------------+-----------+-----------|
| DEFAULT_1f26c_00000 | PENDING  |       |           16 | 0.000280997 |       256 |        32 |
| DEFAULT_1f26c_00001 | PENDING  |       |            4 | 0.00014828  |        32 |        16 |
| DEFAULT_1f26c_00002 | PENDING  |       |            4 | 0.0027054

  2%|▏         | 2720768/170498071 [00:04<03:19, 840275.75it/s]


== Status ==
Current time: 2021-11-18 12:25:15 (running for 00:00:11.22)
Memory usage on this node: 13.0/62.5 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 4.0/28 CPUs, 2.0/2 GPUs, 0.0/33.67 GiB heap, 0.0/16.83 GiB objects (0.0/1.0 accelerator_type:TITAN)
Result logdir: /home/milky/ray_results/DEFAULT_2021-11-18_12-25-04
Number of trials: 10/10 (8 PENDING, 2 RUNNING)
+---------------------+----------+--------------------+--------------+-------------+-----------+-----------+
| Trial name          | status   | loc                |   batch_size |          lr |   nodes_1 |   nodes_2 |
|---------------------+----------+--------------------+--------------+-------------+-----------+-----------|
| DEFAULT_1f26c_00000 | RUNNING  | 192.168.0.16:21284 |           16 | 0.000280997 |       256 |        32 |
| DEFAULT_1f26c_00001 | RUNNING  | 192.168.0.16:21281 |            4 | 0.00014828  |        32 |

  4%|▍         | 6853632/170498071 [00:09<04:13, 646437.42it/s]


...

== Status ==
Current time: 2021-11-18 12:34:02 (running for 00:08:58.32)
Memory usage on this node: 13.3/62.5 GiB
Using AsyncHyperBand: num_stopped=2
Bracket: Iter 8.000: -1.4081315021395684 | Iter 4.000: -1.598354333382845 | Iter 2.000: -1.7860041932940482 | Iter 1.000: -1.8784815873384475
Resources requested: 4.0/28 CPUs, 2.0/2 GPUs, 0.0/33.67 GiB heap, 0.0/16.83 GiB objects (0.0/1.0 accelerator_type:TITAN)
Result logdir: /home/milky/ray_results/DEFAULT_2021-11-18_12-25-04
Number of trials: 10/10 (6 PENDING, 2 RUNNING, 2 TERMINATED)
+---------------------+------------+--------------------+--------------+-------------+-----------+-----------+---------+------------+----------------------+
| Trial name          | status     | loc                |   batch_size |          lr |   nodes_1 |   nodes_2 |    loss |   accuracy |   training_iteration |
|---------------------+------------+--------------------+--------------+-------------+-----------+-----------+---------+------------+------------

 96%|█████████▌| 163234816/170498071 [00:46<00:01, 3715835.09it/s]


== Status ==
Current time: 2021-11-18 12:34:07 (running for 00:09:03.35)
Memory usage on this node: 13.6/62.5 GiB
Using AsyncHyperBand: num_stopped=2
Bracket: Iter 8.000: -1.4081315021395684 | Iter 4.000: -1.598354333382845 | Iter 2.000: -1.7860041932940482 | Iter 1.000: -1.8784815873384475
Resources requested: 4.0/28 CPUs, 2.0/2 GPUs, 0.0/33.67 GiB heap, 0.0/16.83 GiB objects (0.0/1.0 accelerator_type:TITAN)
Result logdir: /home/milky/ray_results/DEFAULT_2021-11-18_12-25-04
Number of trials: 10/10 (6 PENDING, 2 RUNNING, 2 TERMINATED)
+---------------------+------------+--------------------+--------------+-------------+-----------+-----------+---------+------------+----------------------+
| Trial name          | status     | loc                |   batch_size |          lr |   nodes_1 |   nodes_2 |    loss |   accuracy |   training_iteration |
|---------------------+------------+--------------------+--------------+-------------+-----------+-----------+---------+------------+------------

100%|█████████▉| 170034176/170498071 [00:48<00:00, 3923827.43it/s]
170499072it [00:48, 3499124.32it/s]                               


(ImplicitFunc pid=21276) Extracting ./data/cifar-10-python.tar.gz to ./data
(ImplicitFunc pid=21276) Files already downloaded and verified
== Status ==
Current time: 2021-11-18 12:34:12 (running for 00:09:08.36)
Memory usage on this node: 13.8/62.5 GiB
Using AsyncHyperBand: num_stopped=2
Bracket: Iter 8.000: -1.4081315021395684 | Iter 4.000: -1.598354333382845 | Iter 2.000: -1.7860041932940482 | Iter 1.000: -1.8784815873384475
Resources requested: 4.0/28 CPUs, 2.0/2 GPUs, 0.0/33.67 GiB heap, 0.0/16.83 GiB objects (0.0/1.0 accelerator_type:TITAN)
Result logdir: /home/milky/ray_results/DEFAULT_2021-11-18_12-25-04
Number of trials: 10/10 (6 PENDING, 2 RUNNING, 2 TERMINATED)
+---------------------+------------+--------------------+--------------+-------------+-----------+-----------+---------+------------+----------------------+
| Trial name          | status     | loc                |   batch_size |          lr |   nodes_1 |   nodes_2 |    loss |   accuracy |   training_iteration |
|-----

### Report the results 
* ```get_best__trial()``` methods returns an object that contains information about the best trial. 
* print out the hyperparameter settings that yielded the best results

In [ ]:
best_trial = result.get_best_trial(  "loss", "min", "last")

print(f"Best trial config: {best_trial.config}")
print(f"Best trial final validation loss: {best_trial.last_result["loss"]}")
print(f"Best trial final validation accuracy: {best_trial.last_result["accuracy"]}")